In [0]:
import numpy as np
import pandas as pd

# Input data files are available in the "data/" directory.
# For example, running this will list the files in the input directory
from subprocess import check_output
print(check_output(["ls"]).decode("utf8"))

label_int_to_str_mapping.csv
sample_data
sample_submission.csv
test_images.npy
train_images.npy
train_labels.csv
winter2020-mais-202.zip



In [0]:
!unzip /content/winter2020-mais-202.zip
!ls

Archive:  /content/winter2020-mais-202.zip
  inflating: label_int_to_str_mapping.csv  
  inflating: sample_submission.csv   
  inflating: test_images.npy         
  inflating: train_images.npy        
  inflating: train_labels.csv        
label_int_to_str_mapping.csv  test_images.npy	winter2020-mais-202.zip
sample_data		      train_images.npy
sample_submission.csv	      train_labels.csv


In [0]:
y_train = pd.read_csv("train_labels.csv").label.values
y_test = pd.read_csv("sample_submission.csv").label.values


In [0]:
# Import libraries
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# # Load training and test data into dataframes
data_train = np.load('train_images.npy')
data_test = np.load('test_images.npy')

# # X forms the training images, and y forms the training labels
# X = np.array(data_train.iloc[:, 1:])
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Here I split original training data to sub-training (80%) and validation data (20%)
X_train, X_val, y_train, y_val = train_test_split(data_train, y_train, test_size=0.2, random_state=13)

# X_test forms the test images, and y_test forms the test labels
X_test = np.load('test_images.npy')



Using TensorFlow backend.


In [0]:
print(X_train.shape)
#Processing data
# Each image's dimension is 28 x 28
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# Prepare the training images
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_train /= 255

# Prepare the test images
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_test = X_test.astype('float32')
X_test /= 255
print(X_test.shape)
# Prepare the validation images
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
X_val = X_val.astype('float32')
X_val /= 255

(40000, 28, 28)
(20000, 28, 28, 1)


In [0]:
# Import Keras libraries
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

cnn1 = Sequential()
cnn1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
cnn1.add(MaxPooling2D(pool_size=(2, 2)))
cnn1.add(Dropout(0.2))

cnn1.add(Flatten())

cnn1.add(Dense(128, activation='relu'))
cnn1.add(Dense(10, activation='softmax'))







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
cnn1.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [0]:
cnn1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 13, 13, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 5408)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               692352    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1290      
Total params: 693,962
Trainable params: 693,962
Non-trainable params: 0
________________________________________________

In [0]:
print(X_train.shape)
y_train = y_train.reshape(y_train.shape[0],10)
print(y_train.shape)
histroy1 = cnn1.fit(X_train, y_train, batch_size=256, epochs=20, verbose=1, validation_data=(X_val, y_val))

(40000, 28, 28, 1)
(40000, 10)
Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 22s 557us/step - loss: 0.1712 - acc: 0.9387 - val_loss: 0.4427 - val_acc: 0.8533
Epoch 2/20
40000/40000 [==============================] - 22s 560us/step - loss: 0.1626 - acc: 0.9403 - val_loss: 0.4407 - val_acc: 0.8555
Epoch 3/20
40000/40000 [==============================] - 22s 560us/step - loss: 0.1568 - acc: 0.9431 - val_loss: 0.4576 - val_acc: 0.8573
Epoch 4/20
40000/40000 [==============================] - 22s 554us/step - loss: 0.1495 - acc: 0.9468 - val_loss: 0.4577 - val_acc: 0.8580
Epoch 5/20
40000/40000 [==============================] - 22s 552us/step - loss: 0.1452 - acc: 0.9483 - val_loss: 0.4477 - val_acc: 0.8587
Epoch 6/20
40000/40000 [==============================] - 22s 548us/step - loss: 0.1401 - acc: 0.9499 - val_loss: 0.4513 - val_acc: 0.8593
Epoch 7/20
40000/40000 [==============================] - 22s 550us/step - loss: 0.13

In [0]:
# X_test = X_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
score1 = cnn1.evaluate(X_test, y_test, verbose=1)
print('Test loss: ', score1[0])
print('Test accuracy: ', score1[1])

20000/20000 [==============================] - 5s 238us/step
Test loss:  9.88024481048584
Test accuracy:  0.1011


In [0]:
test_images = X_test.reshape((-1, 28, 28, 1))
y_test = cnn1.predict_classes(test_images)

df_test = pd.read_csv('sample_submission.csv')
df_test['label'] = y_test
df_test.to_csv('submission4.csv', index=False)